### Connect to DuckDB and Configure S3/MinIO Access

The following cell initializes a DuckDB connection backed by a local database file and enables the `httpfs` extension to read and write data stored in MinIO via the S3 API. It sets the MinIO endpoint, disables SSL, applies path-style addressing, and loads S3 access credentials from environment variables.

In [1]:
import os
import duckdb

os.makedirs("/opt/de_project/warehouse", exist_ok=True)
db_path = "/opt/de_project/warehouse/taxi.duckdb"
con = duckdb.connect(db_path)

# Enable S3/MinIO access via DuckDB's httpfs extension
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

# Configure DuckDB to connect to MinIO
con.execute("SET s3_endpoint = 'minio:9000';")
con.execute("SET s3_use_ssl = false;")
con.execute("SET s3_url_style = 'path';")

# Credentials
con.execute(f"SET s3_access_key_id = '{os.environ['AWS_ACCESS_KEY_ID']}';")
con.execute(f"SET s3_secret_access_key = '{os.environ['AWS_SECRET_ACCESS_KEY']}';")

con

In [2]:
row_count = con.execute("SELECT COUNT(*) FROM taxi.taxi.trips_prepared;").fetchone()[0]
row_count

2928254

### Sample 100 rows from the prepared Taxi Table

The next cell queries a subset of the prepared dataset from DuckDB, retrieves the results as a Pandas DataFrame, and displays the first few rows for inspection.

In [3]:
q = """
SELECT
    *
FROM taxi.taxi.trips_prepared
LIMIT 100;
"""
df = con.execute(q).fetchdf()
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,total_amount,congestion_surcharge,Airport_fee,pickup_hour,dropoff_date,trip_duration_min,avg_mph,month,pickup_date,year
0,2,2002-12-31 22:59:39,2002-12-31 23:05:41,1,0.63,1,N,170,170,3,...,10.50,2.5,0.0,22,2002-12-31,6.033333,6.265193,01,2002-12-31,2024
1,2,2009-01-01 23:58:40,2009-01-02 00:01:40,1,0.46,1,N,137,264,2,...,9.40,0.0,0.0,23,2009-01-02,3.000000,9.200000,01,2009-01-01,2024
2,2,2009-01-01 00:24:09,2009-01-01 01:13:00,2,10.88,1,N,138,264,2,...,68.29,0.0,0.0,0,2009-01-01,48.850000,13.363357,01,2009-01-01,2024
3,2,2009-01-01 23:30:39,2009-01-02 00:01:39,1,10.99,1,N,237,264,2,...,50.00,0.0,0.0,23,2009-01-02,31.000000,21.270968,01,2009-01-01,2024
4,2,2023-12-31 23:56:46,2024-01-01 00:12:06,2,2.38,1,N,236,142,1,...,21.60,2.5,0.0,23,2024-01-01,15.333333,9.313043,01,2023-12-31,2024


In [5]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee', 'pickup_hour',
       'dropoff_date', 'trip_duration_min', 'avg_mph', 'month', 'pickup_date',
       'year'],
      dtype='object')

### Descriptive statistics of the sampled dataset

The following cell computes summary statistics for all numeric columns in the DataFrame, allowing you to quickly inspect distributions, ranges, and potential data quality issues.


In [6]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,...,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,pickup_hour,dropoff_date,trip_duration_min,avg_mph,pickup_date,year
count,100.000000,100,100,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,...,100.0,100.000000,100.000000,100.000000,100.000000,100,100.000000,100.000000,100,100.0
mean,1.680000,2023-05-04 23:59:16.580000,2023-05-05 00:15:21.180000,1.590000,3.229200,1.050000,169.030000,168.040000,1.29000,19.561000,...,1.0,28.301000,2.325000,0.087500,2.980000,2023-05-04 22:33:36,16.076667,11.061296,2023-05-04 20:24:00,2024.0
min,1.000000,2002-12-31 22:59:39,2002-12-31 23:05:41,0.000000,0.000000,1.000000,4.000000,4.000000,1.00000,3.000000,...,1.0,8.000000,0.000000,0.000000,0.000000,2002-12-31 00:00:00,0.316667,0.000000,2002-12-31 00:00:00,2024.0
25%,1.000000,2024-01-01 00:14:46,2024-01-01 00:27:21.500000,1.000000,0.962500,1.000000,138.000000,113.000000,1.00000,8.600000,...,1.0,16.265000,2.500000,0.000000,0.000000,2024-01-01 00:00:00,6.662500,6.572172,2024-01-01 00:00:00,2024.0
50%,2.000000,2024-01-01 00:32:47,2024-01-01 00:46:18.500000,1.000000,1.820000,1.000000,161.000000,166.500000,1.00000,14.200000,...,1.0,22.850000,2.500000,0.000000,0.000000,2024-01-01 00:00:00,13.408333,9.060872,2024-01-01 00:00:00,2024.0
75%,2.000000,2024-01-01 00:45:39.500000,2024-01-01 01:01:06.750000,2.000000,3.650000,1.000000,236.000000,236.250000,2.00000,26.975000,...,1.0,33.375000,2.500000,0.000000,0.000000,2024-01-01 00:00:00,22.750000,12.345652,2024-01-01 00:00:00,2024.0
max,2.000000,2024-01-01 00:58:18,2024-01-01 01:42:04,6.000000,23.900000,5.000000,263.000000,265.000000,4.00000,120.000000,...,1.0,127.940000,2.500000,1.750000,23.000000,2024-01-01 00:00:00,51.583333,41.285988,2024-01-01 00:00:00,2024.0
std,0.468826,NaN,NaN,1.015983,3.970753,0.411329,64.954911,68.790403,0.53739,16.922138,...,0.0,19.489247,0.641081,0.383325,7.748548,NaN,11.706544,7.761864,NaN,0.0


### Data Quality Checks

We validate general data consistency by inspecting missing values, numeric ranges,
and timestamp coherence. This ensures the Prepared layer is trustworthy before
downstream modeling or BI use.

In [7]:
df.isna().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
Airport_fee              0
pickup_hour              0
dropoff_date             0
trip_duration_min        0
avg_mph                  0
month                    0
pickup_date              0
year                     0
dtype: int64

### Temporal Coverage Validation

Confirm whether the dataset covers the full expected day/hour range.  
This helps detect missing ingestion or partition issues.

In [41]:
con.execute("""
SELECT 
    MIN(pickup_date) AS min_date,
    MAX(pickup_date) AS max_date,
    COUNT(DISTINCT pickup_hour) AS distinct_hours
FROM taxi.taxi.trips_prepared;
""").fetchdf()

,min_date,max_date,distinct_hours
0,2002-12-31,2024-02-01,24


### Hourly aggregation

This query aggregates trips by pickup date and hour, computing trip counts and average metrics. The resulting DataFrame preview below helps confirm that the transformation and DuckDB ingestion produced coherent time-based features.

In [8]:
q = """
SELECT
    pickup_date,
    pickup_hour,
    COUNT(*) AS trips,
    AVG(trip_distance) AS avg_distance_miles,
    AVG(total_amount) AS avg_total_amount
FROM taxi.taxi.trips_prepared
GROUP BY 1, 2
ORDER BY 1, 2
LIMIT 100;
"""
df_hourly = con.execute(q).fetchdf()
df_hourly.head()

,pickup_date,pickup_hour,trips,avg_distance_miles,avg_total_amount
0,2002-12-31,22,1,0.63000,10.500000
1,2009-01-01,0,1,10.88000,68.290000
2,2009-01-01,23,2,5.72500,29.700000
3,2023-12-31,23,10,2.60100,22.462000
4,2024-01-01,0,6523,2.94086,27.438436
